In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install -q rank-bm25 underthesea chromadb
!pip install -q google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 8.2 MB/s eta 

In [5]:
from rank_bm25 import BM25Okapi
from underthesea import word_tokenize
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
from typing import List
from google import genai

import json
import chromadb
import torch

In [6]:
from google.colab import userdata
Gemini_API_key = userdata.get('Gemini_API_key')

gemini_client = genai.Client(api_key=Gemini_API_key)

In [7]:
# test:
resp = gemini_client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Test kết nối"
)
print(resp.text)


Chào bạn! Tôi đã nhận được tin nhắn của bạn. Kết nối tốt.

Bạn cần tôi giúp gì không?


# 1. RAG Flow QUY CHẾ TỔ CHỨC VÀ QUẢN LÝ ĐÀO TẠO TRÌNH ĐỘ ĐẠI HỌC

In [8]:
# Load chunks
with open("/content/drive/MyDrive/NLP/RAG_final/Chunking/qd_1830_chunks_debug.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

bm25_corpus_texts = [c["text"] for c in chunks]
bm25_corpus_tokens = [word_tokenize(t.lower()) for t in bm25_corpus_texts]

bm25 = BM25Okapi(bm25_corpus_tokens)

# Để mapping ngược index -> chunk metadata
bm25_chunk_ids = [c["chunk_id"] for c in chunks]
bm25_chunk_meta = {c["chunk_id"]: c for c in chunks}

In [9]:
# sparse_search
def bm25_search(query: str, top_k: int = 30):
    q_tokens = word_tokenize(query.lower())
    scores = bm25.get_scores(q_tokens)  # 1D array

    # Lấy top_k index theo score
    import numpy as np
    top_idx = np.argsort(scores)[::-1][:top_k]

    results = []
    for rank, idx in enumerate(top_idx):
        chunk_id = bm25_chunk_ids[idx]
        chunk = bm25_chunk_meta[chunk_id]
        results.append({
            "chunk_id": chunk_id,
            "score": float(scores[idx]),
            "rank": rank,
            "source": "sparse",
            "text": chunk["text"],
            "metadata": {
                "doc_id": chunk["doc_id"],
                "chapter_title": chunk["chapter_title"],
                "article_title": chunk["article_title"],
                "chunk_index": chunk["chunk_index"],
                "chunk_type": chunk["chunk_type"],
            }
        })
    return results


In [10]:
# load model & chroma persistent client lúc start app
dense_model = SentenceTransformer("AITeamVN/Vietnamese_Embedding")
dense_model.max_seq_length = 2048

chroma_client = chromadb.PersistentClient(path="/content/drive/MyDrive/NLP/RAG_final/Embedding/chroma_qd1830")
collection = chroma_client.get_collection("qd_1830")

# Dense retriever: Chroma + Vietnamese_Embedding
def dense_search(query: str, top_k: int = 30):
    q_emb = dense_model.encode([query], normalize_embeddings=True).tolist()

    res = collection.query(
        query_embeddings=q_emb,
        n_results=top_k
    )

    results = []
    for i in range(len(res["ids"][0])):
        results.append({
            "chunk_id": res["ids"][0][i],
            "score": float(res["distances"][0][i]),  # khoảng cách, càng nhỏ càng tốt
            "rank": i,
            "source": "dense",
            "text": res["documents"][0][i],
            "metadata": res["metadatas"][0][i],
        })
    return results


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [11]:
# Hybrid retrieval + Reciprocal Rank Fusion (RRF)

def rrf_fusion(dense_results, sparse_results, k: int = 60, top_k: int = 20):
    # map chunk_id -> rank cho từng nguồn
    dense_ranks = {r["chunk_id"]: r["rank"] for r in dense_results}
    sparse_ranks = {r["chunk_id"]: r["rank"] for r in sparse_results}

    # union tất cả chunk_id
    all_ids = set(dense_ranks.keys()) | set(sparse_ranks.keys())

    fused = {}
    for cid in all_ids:
        score = 0.0
        if cid in dense_ranks:
            score += 1.0 / (k + dense_ranks[cid])
        if cid in sparse_ranks:
            score += 1.0 / (k + sparse_ranks[cid])

        fused[cid] = score

    # sort theo fused score (desc)
    sorted_ids = sorted(all_ids, key=lambda x: fused[x], reverse=True)[:top_k]

    # build list kết quả cuối
    # cần lookup text + metadata từ 1 trong 2 nguồn (ưu tiên dense hoặc sparse)
    result_map = {}
    for r in dense_results + sparse_results:
        cid = r["chunk_id"]
        if cid not in result_map:
            result_map[cid] = r

    final_results = []
    for i, cid in enumerate(sorted_ids):
        r = result_map[cid]
        rrf_item = {
            "chunk_id": cid,
            "rrf_score": fused[cid],
            "rank": i,
            "text": r["text"],
            "metadata": r["metadata"],
            "sources": []  # đánh dấu nó đến từ đâu
        }
        srcs = []
        if cid in dense_ranks:
            srcs.append("dense")
        if cid in sparse_ranks:
            srcs.append("sparse")
        rrf_item["sources"] = srcs
        final_results.append(rrf_item)

    return final_results


In [12]:
def hybrid_retrieve(query: str, dense_k=30, sparse_k=30, fused_k=20):
    dense_res = dense_search(query, top_k=dense_k)
    sparse_res = bm25_search(query, top_k=sparse_k)
    fused_res = rrf_fusion(dense_res, sparse_res, top_k=fused_k)
    return fused_res

In [13]:
def call_llm_with_gemini(gemini_client, prompt, model_name="gemini-2.5-flash"):
    response = gemini_client.models.generate_content(
        model=model_name,
        contents=prompt,
    )
    return (response.text or "").strip()


In [14]:
def generate_multi_queries(
    llm_client: genai.Client,
    user_query: str,
    num_queries: int = 3,
    model_name: str = "gemini-2.5-flash",
) -> List[str]:
    """
    Sinh ra num_queries câu truy vấn khác nhau từ 1 user_query,
    dùng Gemini Flash (gemini-2.5-flash).

    llm_client: genai.Client đã được khởi tạo sẵn.
    """

    prompt = f"""
Bạn là trợ lý tìm kiếm văn bản quy chế đào tạo của TDTU.
Hãy viết {num_queries} biến thể truy vấn khác nhau, ngắn gọn, bằng tiếng Việt,
giúp tìm đúng Điều / Chương liên quan trong quy chế từ câu hỏi gốc sau:

Câu hỏi gốc: "{user_query}"

Yêu cầu:
- Không trả lời câu hỏi.
- Mỗi dòng chỉ một câu hỏi.
- Không đánh số, không thêm giải thích.
"""

    text = call_llm_with_gemini(llm_client, prompt, model_name)
    queries = [line.strip() for line in text.splitlines() if line.strip()]
    return queries[:num_queries]


In [15]:
def multiquery_hybrid_retrieve(user_query: str, llm_client, num_queries=3):
    rewrites = generate_multi_queries(llm_client, user_query, num_queries=num_queries)
    all_queries = [user_query] + rewrites

    # cho mỗi query: hybrid_retrieve -> list chunk
    per_query_results = []
    for q in all_queries:
        fused = hybrid_retrieve(q, dense_k=20, sparse_k=20, fused_k=20)
        per_query_results.append(fused)

    # giờ per_query_results là list[list[result]]
    # dùng RRF thêm 1 lần: rank theo xuất hiện + rank trong từng list
    k = 60
    scores = {}
    meta_map = {}

    for res_list in per_query_results:
        for r in res_list:
            cid = r["chunk_id"]
            rank = r["rank"]
            if cid not in scores:
                scores[cid] = 0.0
                meta_map[cid] = r
            scores[cid] += 1.0 / (k + rank)

    sorted_ids = sorted(scores.keys(), key=lambda x: scores[x], reverse=True)[:20]

    final = []
    for i, cid in enumerate(sorted_ids):
        r = meta_map[cid]
        final.append({
            "chunk_id": cid,
            "score": scores[cid],
            "rank": i,
            "text": r["text"],
            "metadata": r["metadata"]
        })
    return final


In [16]:
RERANK_MODEL_NAME = "AITeamVN/Vietnamese_Reranker"
RERANK_MAX_LEN = 2048

device = "cuda" if torch.cuda.is_available() else "cpu"

rerank_tokenizer = AutoTokenizer.from_pretrained(RERANK_MODEL_NAME)
rerank_model = AutoModelForSequenceClassification.from_pretrained(RERANK_MODEL_NAME)
rerank_model.to(device)
rerank_model.eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(8194, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, o

In [17]:
def rerank_with_vietnamese_reranker(
    query: str,
    candidates: list,
    top_k: int = 10,
):
    """
    candidates: list các dict, mỗi dict có ít nhất key 'text' (và metadata khác).
    Trả về list candidates đã sort theo score giảm dần, kèm thêm 'ce_score'.
    """

    if not candidates:
        return []

    # 1) Chuẩn bị cặp (query, passage)
    pairs = [[query, c["text"]] for c in candidates]

    # 2) Tokenize batch
    inputs = rerank_tokenizer(
        pairs,
        padding=True,
        truncation=True,
        max_length=RERANK_MAX_LEN,
        return_tensors="pt",
    ).to(device)

    # 3) Chạy model lấy score
    with torch.no_grad():
        outputs = rerank_model(**inputs, return_dict=True)
        # logits shape: [batch_size, 1] → flatten
        scores = outputs.logits.view(-1).float().cpu().tolist()

    # 4) Gán score vào candidates và sort
    for c, s in zip(candidates, scores):
        c["ce_score"] = float(s)

    reranked = sorted(candidates, key=lambda x: x["ce_score"], reverse=True)
    return reranked[:top_k]


In [18]:
def retrieve_for_rag_multiquery(query: str, llm_client, num_queries: int = 3, top_k: int = 8):
    # sinh thêm các query
    rewrites = generate_multi_queries(llm_client, query, num_queries=num_queries)
    all_queries = [query] + rewrites

    # cho mỗi query, chạy hybrid
    all_results = []
    for q in all_queries:
        res = hybrid_retrieve(q, dense_k=20, sparse_k=20, fused_k=20)
        all_results.append(res)

    # RRF lần hai để gộp multi-query
    k = 60
    scores = {}
    meta_map = {}

    for res_list in all_results:
        for r in res_list:
            cid = r["chunk_id"]
            rank = r["rank"]
            if cid not in scores:
                scores[cid] = 0.0
                meta_map[cid] = r
            scores[cid] += 1.0 / (k + rank)

    sorted_ids = sorted(scores.keys(), key=lambda x: scores[x], reverse=True)[:top_k]

    final = []
    for i, cid in enumerate(sorted_ids):
        r = meta_map[cid]
        final.append({
            "chunk_id": cid,
            "score": scores[cid],
            "rank": i,
            "text": r["text"],
            "metadata": r["metadata"],
        })
    return final


In [19]:
def build_context_from_chunks(chunks, max_chars: int = 4000) -> str:
    """
    chunks: list các item từ retrieve_for_rag(...)
    Trả về 1 string gộp nhiều đoạn, mỗi đoạn có tiêu đề Điều/Chương.
    """
    pieces = []
    total = 0

    for c in chunks:
        meta = c["metadata"]
        chapter = meta.get("chapter_title", "")
        article = meta.get("article_title", "")
        header = f"{chapter} / {article}".strip(" /")
        text = c["text"].strip()

        block = f"---\nNguồn: {header}\nNội dung:\n{text}\n"
        if total + len(block) > max_chars:
            break
        pieces.append(block)
        total += len(block)

    return "\n".join(pieces)


In [20]:
def build_rag_prompt(user_query: str, context: str) -> str:
    return f"""
Bạn là trợ lý hỗ trợ sinh viên và giảng viên hiểu quy chế đào tạo của Trường Đại học Tôn Đức Thắng.

Dưới đây là trích đoạn từ các văn bản quy chế (có thể đã được rút gọn):

[CONTEXT BẮT ĐẦU]
{context}
[CONTEXT KẾT THÚC]

Nhiệm vụ:
- Trả lời câu hỏi dưới đây **chỉ dựa trên thông tin trong CONTEXT**.
- Nếu thông tin trong CONTEXT không đủ để trả lời, hãy nói rõ là "Trong tài liệu hiện có, không thấy quy định cụ thể về vấn đề này" và gợi ý người dùng kiểm tra thêm trong quy chế đầy đủ.
- Khi trả lời, hãy trích dẫn Điều/Chương tương ứng (nếu context có), ví dụ: "Theo Điều 24, Chương IV,...".

Câu hỏi: "{user_query}"

Trả lời bằng tiếng Việt, mạch lạc, dễ hiểu.
"""


In [21]:
def answer_question(
    query: str,
    llm_client: genai.Client,
    top_k: int = 8,
) -> dict:
    """
    Trả về dict gồm:
      - answer: câu trả lời của Gemini
      - used_chunks: list metadata về các chunk đã dùng (để debug / show nguồn)
    """

    # 1) Retrieve
    retrieved = retrieve_for_rag_multiquery(query, llm_client, num_queries=3, top_k=top_k)
    retrieved = rerank_with_vietnamese_reranker(query, retrieved, top_k=top_k)

    # 2) Build context string
    context_str = build_context_from_chunks(retrieved, max_chars=4000)

    # 3) Build prompt
    prompt = build_rag_prompt(query, context_str)

    # 4) Gọi Gemini
    answer = call_llm_with_gemini(llm_client, prompt, model_name="gemini-2.5-flash")

    # 5) Trả về cả câu trả lời và metadata của context
    used_chunks = [
        {
            "chunk_id": c["chunk_id"],
            "doc_id": c["metadata"].get("doc_id"),
            "chapter_title": c["metadata"].get("chapter_title"),
            "article_title": c["metadata"].get("article_title"),
            "chunk_index": c["metadata"].get("chunk_index"),
            "score": c.get("score") or c.get("rrf_score"),
        }
        for c in retrieved
    ]

    return {
        "answer": answer,
        "used_chunks": used_chunks,
    }


In [22]:
result = answer_question(
    "Chương trình đào tạo của đại học Tôn Đức Thắng được tính theo đơn vị gì ?",
    llm_client=gemini_client,   # dùng Gemini client, không phải chroma_client
    top_k=8,
)


print("===== TRẢ LỜI =====")
print(result["answer"])
print("\n===== NGUỒN CONTEXT =====")
for c in result["used_chunks"]:
    print(c)

===== TRẢ LỜI =====
Chào bạn,

Theo thông tin trong tài liệu, **Chương trình đào tạo (CTĐT) của Trường Đại học Tôn Đức Thắng được xây dựng theo đơn vị tín chỉ**.

(Theo Điều 2, Chương I. NHỮNG QUY ĐỊNH CHUNG)

===== NGUỒN CONTEXT =====
{'chunk_id': 'qd_1830_2021_art_1', 'doc_id': 'qd_1830_2021', 'chapter_title': 'Chương I. NHỮNG QUY ĐỊNH CHUNG', 'article_title': 'Điều 2. Chương trình đào tạo', 'chunk_index': 0, 'score': 0.04750158127767236}
{'chunk_id': 'qd_1830_2021_art_97', 'doc_id': 'qd_1830_2021', 'chapter_title': 'Chương III. ĐÁNH GIÁ KẾT QUẢ HỌC TẬP', 'article_title': 'Điều 27. Cách tính điểm trung bình học kỳ và tích lũy', 'chunk_index': 1, 'score': 0.042056023634971004}
{'chunk_id': 'qd_1830_2021_art_0', 'doc_id': 'qd_1830_2021', 'chapter_title': 'Chương I. NHỮNG QUY ĐỊNH CHUNG', 'article_title': 'Điều 1. Phạm vi và đối tượng áp dụng', 'chunk_index': 0, 'score': 0.045526695526695526}
{'chunk_id': 'qd_1830_2021_art_8', 'doc_id': 'qd_1830_2021', 'chapter_title': 'Chương I. NHỮNG 